In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from zip_code import fetch_census_data as fetch

In [2]:
# Read csv
df = pd.read_csv('city-state_zip-codes.csv')
df

,City-State,Zip_Code
0,Clarksburg-MD,['20871']
1,Boyds-MD,['20841']
2,Rockville-MD,"['20850', '20851', '20852', '20853']"
3,Germantown-MD,"['20874', '20876']"
4,Gaithersburg-MD,"['20877', '20878', '20879', '20882']"
5,Frederick-MD,"['21701', '21702', '21703', '21704']"
6,Ashburn-VA,"['20147', '20148']"
7,Belmont-VA,['23014']
8,Sterling-VA,"['20164', '20165', '20166']"


In [3]:
# Check for columns name and info
df.columns, df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   City-State  9 non-null      object
 1   Zip_Code    9 non-null      object
dtypes: object(2)
memory usage: 272.0+ bytes


(Index(['City-State', 'Zip_Code'], dtype='object'), None)

In [4]:
# Created this function to clean the strings
def trim_string(word):
    """Removes the first and last characters from a string."""
    if len(word) <= 2:
        return ''  # Return an empty string if word is too short
    return word[1:-1]

In [5]:
# Removing brackets by joining the values into a single string
df['Zip_Code'] = df['Zip_Code'].apply(lambda x: trim_string(x))
df

,City-State,Zip_Code
0,Clarksburg-MD,'20871'
1,Boyds-MD,'20841'
2,Rockville-MD,"'20850', '20851', '20852', '20853'"
3,Germantown-MD,"'20874', '20876'"
4,Gaithersburg-MD,"'20877', '20878', '20879', '20882'"
5,Frederick-MD,"'21701', '21702', '21703', '21704'"
6,Ashburn-VA,"'20147', '20148'"
7,Belmont-VA,'23014'
8,Sterling-VA,"'20164', '20165', '20166'"


In [6]:
# If there are multiple zip codes, split each into a different row
explode_df = df.assign(Zip_Code=df['Zip_Code'].str.split(", ")).explode('Zip_Code').reset_index(drop=True)
explode_df['Zip_Code'] = explode_df['Zip_Code'].apply(lambda x: trim_string(x))
explode_df

,City-State,Zip_Code
0,Clarksburg-MD,20871
1,Boyds-MD,20841
2,Rockville-MD,20850
3,Rockville-MD,20851
4,Rockville-MD,20852
5,Rockville-MD,20853
6,Germantown-MD,20874
7,Germantown-MD,20876
8,Gaithersburg-MD,20877
9,Gaithersburg-MD,20878


In [7]:
final_df = explode_df.copy()

In [8]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   City-State  22 non-null     object
 1   Zip_Code    22 non-null     object
dtypes: object(2)
memory usage: 480.0+ bytes


In [9]:
# These are the variables I was interested in. For more variables, please look census_variables.csv. 
# This .csv file was created by web scrapping of US census webpage using "census_cariable_scrap.py".
#
variables_dict = {'DP05_0044E': 'One race | Asian', 'DP05_0045E': 'One race | Asian | Asian Indian', 'DP05_0046E': 'One race | Asian | Chinease'}
variables = ','.join(list(variables_dict.keys()))
year = '2022' 

In [10]:
# retireve first zip code in the column
final_df['Zip_Code'].iloc[0]

'20871'

In [11]:
# Cheking if the fetch function is working fine
fetch(year, variables, final_df['Zip_Code'].iloc[0])[1]

['10267', '4626', '1810', '20871']

In [12]:
# Run the function on all zip codes and provide the result in combined column.
# If US census doesn't have a value for a zip code, return it None.
final_df['combined'] = final_df['Zip_Code'].apply(lambda x: fetch(year, variables, x)[1] if fetch(year, variables, x) is not None else None)

Error fetching data: 204


In [13]:
final_df

,City-State,Zip_Code,combined
0,Clarksburg-MD,20871,"[10267, 4626, 1810, 20871]"
1,Boyds-MD,20841,"[4267, 1455, 1837, 20841]"
2,Rockville-MD,20850,"[13467, 2687, 6657, 20850]"
3,Rockville-MD,20851,"[2357, 217, 725, 20851]"
4,Rockville-MD,20852,"[9654, 2070, 2442, 20852]"
5,Rockville-MD,20853,"[3907, 348, 779, 20853]"
6,Germantown-MD,20874,"[10859, 3503, 2723, 20874]"
7,Germantown-MD,20876,"[5279, 1580, 1443, 20876]"
8,Gaithersburg-MD,20877,"[6188, 1108, 1414, 20877]"
9,Gaithersburg-MD,20878,"[17775, 6433, 6687, 20878]"


In [14]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   City-State  22 non-null     object
 1   Zip_Code    22 non-null     object
 2   combined    21 non-null     object
dtypes: object(3)
memory usage: 656.0+ bytes


In [15]:
split_df = final_df.copy()

In [16]:
split_df = split_df.dropna()
split_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21 entries, 0 to 21
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   City-State  21 non-null     object
 1   Zip_Code    21 non-null     object
 2   combined    21 non-null     object
dtypes: object(3)
memory usage: 672.0+ bytes


In [17]:
# Split information from combined and assign the values to their respective columns
split_df['Asian_Total'] = split_df['combined'].apply(lambda x: x[0])
split_df['Asian_Indian'] = split_df['combined'].apply(lambda x: x[1])
split_df['Asian_Chinease'] = split_df['combined'].apply(lambda x: x[2])

In [18]:
# Since we are done with 'combined' column, we can drop this column
df_clean = split_df.drop(columns=['combined'])
df_clean

,City-State,Zip_Code,Asian_Total,Asian_Indian,Asian_Chinease
0,Clarksburg-MD,20871,10267,4626,1810
1,Boyds-MD,20841,4267,1455,1837
2,Rockville-MD,20850,13467,2687,6657
3,Rockville-MD,20851,2357,217,725
4,Rockville-MD,20852,9654,2070,2442
5,Rockville-MD,20853,3907,348,779
6,Germantown-MD,20874,10859,3503,2723
7,Germantown-MD,20876,5279,1580,1443
8,Gaithersburg-MD,20877,6188,1108,1414
9,Gaithersburg-MD,20878,17775,6433,6687


In [19]:
# To create a pivot table based on city-state, I dropped zip code column, becuase during processing the zip code will be 
# added and will not provide any relevant information.
df_no_zip = df_clean.drop(columns=['Zip_Code'])

In [20]:
df_no_zip.columns

Index(['City-State', 'Asian_Total', 'Asian_Indian', 'Asian_Chinease'], dtype='object')

In [21]:
# Created the pivot table and transposed the table.
pivot = pd.pivot_table(df_no_zip, 
                       columns='City-State',
                       values=['Asian_Total', 'Asian_Indian', 'Asian_Chinease'],
                       aggfunc='sum')
pivot = pivot.transpose()
pivot

,Asian_Chinease,Asian_Indian,Asian_Total
City-State,,,
Ashburn-VA,10421524,636618234,1500923822
Boyds-MD,1837,1455,4267
Clarksburg-MD,1810,4626,10267
Frederick-MD,405464205844,1485856331136,1107213036172684
Gaithersburg-MD,1414668753295,11086433751243,6188177753807961
Germantown-MD,27231443,35031580,108595279
Rockville-MD,66577252442779,26872172070348,13467235796543907
Sterling-VA,513597206,154014261877,621246274080
